# Data Science Workshop Project
**Team:** Elad Bilman, Oz Hizkia, Eva Hallermeier, Tzach Cohen

**Problem:** The prediction of in-hospital mortality for admitted patients remain poorly characterized and is biased by
             Doctor's opinion.

**Goal of the project**: We aimed to develop and validate a prediction model for all-cause in-hospital mortality among
                         admitted patients.

## Part 1 - Prologue
### Dataset: Patient Survival Prediction

We choose a dataset from Kaggle  https://www.kaggle.com/datasets/mitishaagarwal/patient

This dataset is a collection of patients that visited the ICU in an hospital. Each row represents a patient, <br>
which in itself is a collection of checkups of the patient 1 hour after his reception and 1 day after his reception, <br>
among other medical examinations. The data also includes the basic characeristics of the patient such as gender, <br>
ethnicity, etc.



#### Import Modules, Libraries and Functions

In [ ]:
%matplotlib inline

# Data Processing
import pandas as pd
import numpy as np

# Machine Learning Library
import sklearn
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

# Analyze Model Tools
import shap
from sklearn.metrics import confusion_matrix

# Visualization Tools
import matplotlib.pylab as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns

from DataprocessingMethods import *
from Model.FinalModel import FinalModel
from Model.ModelModule import DSWorkshopModel
from MissingValuesVisualization import *

# QOL Functions
from UsefullFunctions import load_dataset
from UsefullFunctions import conf_matrix
from UsefullFunctions import plotPR
from UsefullFunctions import plotRoc

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Load Original Dataset
df = load_dataset()
df.head() # Overview of the dataset.

The dataset is composed of 85 columns (features) and we have 91,713 patients. <br> 
We can see that we have an entire empty column (index=83) that we will remove.

## Part 2 - Understanding The Dataset

We have different types of feature: numerical, categorical and binary. Some of the features are data about <br> the medical measures of the patient and some are description of hospital unit and cares he received. <br>

<span style='background:#5c97ff;color:Black'>
In addition, we wrote a complete file which explains all medical features.
</span>

In [ ]:
categorial_features = ["hospital_id", "ethnicity", "gender", "icu_admit_source", 
                       "apache_3j_bodysystem", "apache_2_bodysystem", "icu_stay_type", "icu_type"]

numerical_features = ["age", "bmi","height", "weight", 
                      "pre_icu_los_days", "gcs_eyes_apache","apache_2_diagnosis",
                      "gcs_motor_apache", "gcs_verbal_apache", "heart_rate_apache",
                     "map_apache", "resprate_apache", "temp_apache", "d1_diasbp_max",
                      "d1_diasbp_min","d1_diasbp_noninvasive_max", "d1_diasbp_noninvasive_min",
                      "d1_heartrate_max", "d1_heartrate_min", "d1_mbp_max", "d1_mbp_min", 
                      "d1_mbp_noninvasive_max", "d1_mbp_noninvasive_min", "d1_resprate_max", 
                      "d1_resprate_min","d1_spo2_max", "d1_spo2_min", "d1_sysbp_max", "d1_sysbp_min",
                      "d1_sysbp_noninvasive_max", "d1_sysbp_noninvasive_min", "d1_temp_max", 
                      "d1_temp_min","h1_diasbp_max", "h1_diasbp_min", "h1_diasbp_noninvasive_max", 
                      "h1_diasbp_noninvasive_min","h1_heartrate_max", "h1_heartrate_min", 
                      "h1_mbp_max", "h1_mbp_min","h1_mbp_noninvasive_max", "h1_mbp_noninvasive_min",
                      "h1_resprate_max", "h1_resprate_min","h1_spo2_max", "h1_spo2_min", 
                      "h1_sysbp_max", "h1_sysbp_min","h1_sysbp_noninvasive_max", 
                      "h1_sysbp_noninvasive_min", "d1_glucose_max", "d1_glucose_min",
                      "d1_potassium_max", "d1_potassium_min", "apache_4a_hospital_death_prob", 
                      "apache_4a_icu_death_prob","apache_3j_diagnosis"]


binary_features = ["arf_apache", "gcs_unable_apache", "intubated_apache", 
                   "ventilated_apache", "elective_surgery", "apache_post_operative",
                   "aids", "cirrhosis", "diabetes_mellitus", "hepatic_failure", "immunosuppression",
                   "leukemia", "lymphoma", "solid_tumor_with_metastasis"]

### Part 2.1 - The Problem

**Info About the Prediction Problem:** <br>
We chose to classify wether the patient will "go into very critical state" or not as the dataset have a fetaure of wether the <br>
patient is alive or not.
So our problem is binary classification, if the patient is alive it's marked as 0, otherwise 1.

### Part 2.2 - Thankfully Unbalanced Dataset

In [ ]:
class_prediction = "hospital_death"
unique_labels = np.unique(df[class_prediction])
plt.pie(np.array([len(df[df[class_prediction]==label]) for label in unique_labels ]), 
        labels = list(unique_labels), autopct='%1.1f%%')

As we can see, the difference in quantity of the two classes is exceptionally large, <br>
with the label '1' being far less common. <br>
Due to that, it will be a challenge for the model to predict the label '1', which represents cases that ended <br>
with patients' death.

As such we face the challenge of keeping the model from being biased, since most of the data is labeled with '0' <br>
which will cause the model to easily predict this label, but not the other.

## Part 4 - Sanitizing and Organizing the Dataset
### Part 4.1 - Missing Data

In [ ]:
visualizeMissingValues(df)

In [ ]:
missingValuesDistribution(df)

<span style='background:#ffd500;color:Black'> Most of missing values are numerical. </span>


In [ ]:
df.drop(df.columns[[0,1,83]], axis=1, inplace=True) # Remove useless column

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
featuresWithManyMissingValues = missing_value_df.sort_values(by='percent_missing',ascending=False).head(25)
featuresWithManyMissingValues.head(15)

### Part 4.2 - Deal With Missing Values:
- For Categorical values:
    - We will do one hot encoding so we would not need to deal with missing values. <br>
    We wouldn't include in the clean dataset cells with no categorial match and put a meaningless value.
- For Numerical values:
    - We will fill the cells with the mean of the feature taken <span style='color: #ff0000'> ONLY </span> from the test.
- For Binary values: 
    - We will fill missing values with 0 because most of binary values tell if a patient was given a specific problem <br>
    or a specific treatment, we will assume that if its not written that it hasn't happened.

### Part 4.3 - Informative Data Visualization## 5- Data visualization

#### Feature Distribution

In [ ]:
# Histograms of interesting categorical feature: gender, ethnicity and admission diagnoosis (type of health problem)

columns =  ["gender", "ethnicity","apache_3j_bodysystem"]
fig, axes = plt.subplots(1,3,figsize=(12,5))

n=len(columns)
num_rows = 1
max_bars = 8

for i,variable in enumerate(columns):
    u=min(df[variable].nunique(),max_bars)
    vc = df[variable].value_counts()[:u]
    plt.rcParams.update({'font.size': 18})
    vc.plot(kind='bar',ax=axes[i],title=variable)
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.99, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)

#### Influence of the Age of the Patient on Hospital Death

We know that older patient are at a bigger risk of complications and we can see it clearly on this graph. <br>
The number of deaths is bigger for older patients. <br>
In natural correlation we will also see in the model examination that age is a <b>very</b> important feature.

In [ ]:
dead_patient = df[df["hospital_death"] == 1]
living_patient = df[df["hospital_death"] == 0]

fig,ax=plt.subplots(figsize=(15,10))

#create two histograms
sns.distplot(dead_patient.age, bins = 25, kde = True, label = "dead",ax=ax)
sns.distplot(living_patient.age, bins = 25, kde = True, label = "living",ax=ax)

plt.title('Hospital death Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Count')
plt.legend(loc="upper right")

In [ ]:
df = getBasicDataset()
#prepare data
x_data = df.drop('hospital_death', axis=1)
true_values = df.hospital_death
x_train, x_test, y_train, y_test = train_test_split(x_data, true_values, test_size=0.2, stratify=true_values, shuffle=True)
x_train = fill_missing_num_values_with_mean(x_train)
x_train = fill_missing_values_binary(x_train)
x_test = fill_test_missing_num_values_with_mean(x_test,x_train)
x_test = fill_test_missing_values_binary(x_test, x_train)

all_data = pd.concat([x_train, x_test], axis=0)

## Part 6 - Running Naive Model <br> for Baseline Results and Performance Analysis

We tested a few basic models:
- Random Forest
- Extra Trees
- XGBoost

We get the best result from the XGBoost so we decided to stick with it for our advanced model.
<br>See the basic results:

In [ ]:
#init model
model = DSWorkshopModel(df) 
model.set_split(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

#train mondel
model.train()

#test model
models_predictions, pred_results = model.test()

#results
print(pred_results)

Because we have an unbalanced model we will use "Balanced Accuracy" and "Recall" as our "guiding" metrics.

*Note: Balanced Accuarcy is like the Accuracy metric but the skewness of the metrics.

**Result of the basic model as baseline:**
- Recall around (positive accuracy) 33% so many false negative
- Precision around  67%
- Global Accuracy 93%
- Balanced accuracy around 65%
- Negative accuracy around 98 % 

Because we have only around 10% of death cases in our dataset it means that if we have 90% accuracy <br>
thus we probably always give 0 as prediction let's, verify it.

In [ ]:
cf_matrix = confusion_matrix(y_test.to_numpy(), models_predictions[-1])
conf_matrix(cf_matrix)

As we predicted the biggest problem is that we have <span style='color: #ff0000'> a lot </span> of false
negative predictions so our goal will be to reduce it. <br>

<span style='color: #ff0000'> Important: </span>
<br>
We will prefer to predict the "worst" (practicaly increasing the false positives) such that a patient <br> in danger
will receive more attention and intensive care which maybe can save him.

### Part 6.1 - Analyze Performance of the Basic Model
**Precision Recall-Curve:**
Precision-Recall curves should be used when there is a moderate to large class imbalance.

In [ ]:
y_true = y_test
xgb = model.get_models()[0] #xgboost
predicted_probs = xgb.predict_proba(x_test)
y_score = predicted_probs[:,1]
    
precision, recall, thresholds = sklearn.metrics.precision_recall_curve(y_true, y_score, pos_label=1)
plotPR(precision, recall)

**ROC Plot:**<br>
ROC curves should be used when there are roughly equal numbers of observations for each class which is not our case.

In [ ]:
auc = sklearn.metrics.roc_auc_score(y_true, y_score)
fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_true, y_score)
plotRoc(fpr, tpr, auc)

### Part 6.2 - SHAP Analisys

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(all_data)

#feature importance analysis
shap.summary_plot(shap_values, all_data, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, all_data)

We can see here clearly here which features have the biggest influence on the prediction of the model.
- 1. Hospital Death Probability
- 2. Age
- 3. Minimun Heartrate after 1 day
- 4. Ventilated Apache

All of those features makes sense so we want to capitilize on them in the final model.
Also as we saw in the graph age is a core player in the prediction.

## Part 7 - Advanced Model

### Part 7.1 - Removing Higle Effictive Yet Correlated Feature
In our analysis of the model we noticed two highly correlated features:
- 1. apache_4a_hospital_death_prob
- 2. apache_4a_icu_death_prob

Below we see the value which is the total number of missing values in each one so we have missing values <br>
for same patients in both of the features. <br>

In [ ]:
df[["apache_4a_hospital_death_prob","apache_4a_icu_death_prob"]].dropna().corr()

In [ ]:
poc_dataset = load_dataset()
#they have same quantity of missing values (based on our previous missing value analysis)
#We would like to check if missing values are for same patients
counter = 0

for idx,row in poc_dataset.iterrows():
    if pd.isnull(row["apache_4a_hospital_death_prob"]) and pd.isnull(row["apache_4a_icu_death_prob"]):
        counter+=1

print((counter/poc_dataset.shape[0])*100)

Lets now try to check if they have same values or really close:<br>
- if yes we don't need both of them and we will keep only one, the one that will get better performance. 

In [ ]:
counter = 0

for idx,row in poc_dataset.iterrows():
    if abs(row["apache_4a_hospital_death_prob"] - row["apache_4a_icu_death_prob"])<0.15:
        counter+=1

print((counter/poc_dataset.shape[0])*100)

85% of the values are really close, so it means that we don't really need both.

In [ ]:
# Get Original Dataset
df = getBasicDataset()

# Dropping the Deature
df = df.drop('apache_4a_hospital_death_prob', axis=1)


# Drop the classification column.
x_data = df.drop('hospital_death', axis=1)
true_values = df.hospital_death

### Part 7.2 - Filling Missing Values

In [ ]:
# Split the dataset into train, test.
x_train, x_test, y_train, y_test = train_test_split(x_data, true_values, test_size=0.2,
    stratify=true_values, shuffle=True)
# Fill missing values.
x_train = fill_missing_num_values_with_mean(x_train)
# Fill missing binary values.
x_train = fill_missing_values_binary(x_train)
# Fill missing values.
x_test = fill_test_missing_num_values_with_mean(x_test,x_train)
# Fill missing binary values.
x_test = fill_test_missing_values_binary(x_test, x_train)

### Part 7.3 - Oversampling: Make the Training Set More Balanced

In order to deal with the skewness of the dataset we use an Oversampling algorithm to increase the number of "1" <br>
in our dataset. <br>
We use the RandomOverSampler algorithm we pickes random lines in the class and generates new ones accordingly.<br>
*Note: we use this <span style='color: #ff0000'> ONLY </span> on the train.


In [ ]:
# Getting the oversample model.
oversample = RandomOverSampler(sampling_strategy='minority')
x_np = x_train.to_numpy()
y_np = y_train.to_numpy()
# Oversampling the test.
x_np, y_np = oversample.fit_resample(x_np, y_np)
# Convert back to pandas
x_train = pd.DataFrame(x_np, columns=x_train.columns)
y_train = pd.Series(y_np, name=y_train.name)

### Part 7.4 - Running the Advanced Model

In [ ]:
model = FinalModel(df)
model.set_split(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

#train mondel
model.train()

#test model
models_predictions, pred_results = model.test()

#results
print(pred_results)

**Result of the basic model as baseline:**
- Recall around (positive accuracy) 81% and was 33% in the basic model!
- Balanced accuracy around 77 % and was around 65% in the basic model!
- Negative accuracy and unbalanced accuracy (normal one) decrease


In [ ]:
cf_matrix = confusion_matrix(y_test.to_numpy(), models_predictions[-1])
conf_matrix(cf_matrix)

We can say now that we now predict better death cases which was our goal.

### SHAP on the new model

In [ ]:
y_true = y_test
all_data = pd.concat([x_train, x_test], axis=0)
xgb = model.get_models()[0] #xgboost
predicted_probs = xgb.predict_proba(x_test)
shap.initjs()
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(all_data)

#feature importance analysis
shap.summary_plot(shap_values, all_data, plot_type="bar")

We can see also that we have as expected high correlation between them

### Run with only "apache_4a_icu_death_prob"

In [ ]:
df = getBasicDataset()
df = df.drop('apache_4a_hospital_death_prob', axis=1)
#prepare data
x_data = df.drop('hospital_death', axis=1)
true_values = df.hospital_death
x_train, x_test, y_train, y_test = train_test_split(x_data, true_values, test_size=0.2, stratify=true_values, shuffle=True)
x_train = fill_missing_num_values_with_mean(x_train)
x_train = fill_missing_values_binary(x_train)
x_test = fill_test_missing_num_values_with_mean(x_test,x_train)
x_test = fill_test_missing_values_binary(x_test, x_train)

oversample = RandomOverSampler(sampling_strategy='minority')
x_np = x_train.to_numpy()
y_np = y_train.to_numpy()
x_np, y_np = oversample.fit_resample(x_np, y_np)
# Convert back to pandas
x_train = pd.DataFrame(x_np, columns=x_train.columns)
y_train = pd.Series(y_np, name=y_train.name)


model = FinalModel(df)
model.set_split(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

#train mondel
model.train()

#test model
models_predictions, pred_results = model.test()

#results
print(pred_results)

### Run with only "apache_4a_hospital_death_prob"

In [ ]:
df = getBasicDataset()
df =df.drop('apache_4a_icu_death_prob',axis=1)
#prepare data
x_data = df.drop('hospital_death', axis=1)
true_values = df.hospital_death
x_train, x_test, y_train, y_test = train_test_split(x_data, true_values, test_size=0.2, stratify=true_values, shuffle=True)
x_train = fill_missing_num_values_with_mean(x_train)
x_train = fill_missing_values_binary(x_train)
x_test = fill_test_missing_num_values_with_mean(x_test,x_train)
x_test = fill_test_missing_values_binary(x_test, x_train)

oversample = RandomOverSampler(sampling_strategy='minority')
x_np = x_train.to_numpy()
y_np = y_train.to_numpy()
x_np, y_np = oversample.fit_resample(x_np, y_np)
# Convert back to pandas
x_train = pd.DataFrame(x_np, columns=x_train.columns)
y_train = pd.Series(y_np, name=y_train.name)


model = FinalModel(df)
model.set_split(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

#train mondel
model.train()

#test model
models_predictions, pred_results = model.test()

# Results
print(pred_results)

We get better performance with only "apache_4a_hospital_death_prob" so we will remove for next trials the feature "apache_4a_icu_death_prob"

## 7.2 Oversampling with arrangement of data based on importance of features

Lets try to check which features have  an important impact on the model and have many missing values filed which can give bad data to the model.We will build list of features with high importance and many missing values and we will analyze them and decide how to represent them is the dataset.

In [ ]:
featuresWithManyMissingValues = featuresWithManyMissingValues[['column_name']]

vals= np.abs(shap_values).mean(0)
feature_importance = pd.DataFrame(list(zip(all_data.columns,vals)),columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'],ascending=False,inplace=True)

In [ ]:
most_important_feature = feature_importance['col_name'].tolist()
featuresWithManyMissingValues = featuresWithManyMissingValues['column_name'].tolist()
set1 = set(most_important_feature)
set2 = set(featuresWithManyMissingValues)
newList = list(set1.intersection(set2))
print("Important features with many missing data originally are:\n", newList)

About the feature h1_spo2_max and 'h1_spo2_min': it about saturation: percentage of oxygene in the blood: someone in good health has around 100 % and someone which have pulmonary or very sick can have between 80 and 95%. Lets check the distribution of the data.

In [ ]:
df = getBasicDataset()

dfspo2max = df['h1_spo2_max'].mean()
print("mean 'h1_spo2_max' is ", dfspo2max)
dfspo2min = df['h1_spo2_min'].mean()
print("mean 'h1_spo2_min' is ", dfspo2min)


For those features for saturation we check that mean of saturation based on dataset are ok for filling missing values because normal saturation is higher than around 96% so we can say that maximum can be 98 % and minimum 95 % : those values are acceptable.

About d1_potassium_max and d1_potassium_min: let s try to make the same analysis

In [ ]:
df = getBasicDataset()

dfspo2max = df['d1_potassium_max'].mean()
print("mean 'd1_potassium_max' is ", dfspo2max)
dfspo2min = df['d1_potassium_min'].mean()
print("mean 'd1_potassium_min' is ", dfspo2min)

## Conclusion

We built a model which predict death case for admitted patient at hospital. Our model is sensitive and can predict death even if the patient will probably finish alive but it is preferable this way because we don t want to miss patient that can be not treated as they need in order to stay alive.